<a href="https://colab.research.google.com/github/Athugodage/Hyperskill_transliteration/blob/main/best_transliteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/Hyperskill_transliteration.git

Cloning into 'Hyperskill_transliteration'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 8), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), 64.33 KiB | 1.57 MiB/s, done.


In [2]:
!pip install huggingface_hub
!pip install transformers datasets evaluation sacrebleu Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import pandas as pd

df = pd.read_csv('/content/Hyperskill_transliteration/polish2english.csv')

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=0.078)

In [37]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict({'train': Dataset.from_pandas(train),
             'test': Dataset.from_pandas(test)},
             header=None,
             index_col = False,
             sep=',')

In [26]:
dataset['train'][0]

{'Polish': 'Laurencjusz', 'English': 'Lawrence', '__index_level_0__': 145}

In [15]:
from datasets import load_dataset

In [ ]:



'''dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )'''

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-866682cf83f00415/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
dataset.push_to_hub('marcus2000/polish_names')

TypeError: ignored

In [ ]:
dataset = load_dataset('marcus2000/polish_names')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/64 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/572 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--polish_names-fa6547915a779d8a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("t5-base", use_fast=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [38]:

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_hf = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

In [39]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Polish', 'English', '__index_level_0__'],
        num_rows: 586
    })
    test: Dataset({
        features: ['Polish', 'English', '__index_level_0__'],
        num_rows: 50
    })
    header: None
    index_col: False
    sep: ,
})

In [40]:
max_input_length = 5
max_target_length = 5


def preprocess_function(examples):
    inputs = [ex for ex in examples['Polish']]  # 0 - Текст
    targets = [ex for ex in examples['English']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
     

In [42]:
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
     

Map:   0%|          | 0/586 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [52]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 1

args = Seq2SeqTrainingArguments("polish_transliterator2",
    evaluation_strategy = "epoch",
    learning_rate=3e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)
    

In [53]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_hf)

In [47]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=991200e6aea12f996770497770342a286886edb3c2c699e8506b6b424fc96598
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [48]:
from datasets import load_metric


metric = load_metric("rouge")

In [57]:
import nltk
import numpy as np
from datasets import load_metric

nltk.download("punkt", quiet=True)

metric = load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [58]:
trainer = Seq2SeqTrainer(
    model_hf,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

/content/polish_transliterator2 is already a clone of https://huggingface.co/marcus2000/polish_transliterator2. Make sure you pull the latest changes with `repo.git_pull()`.


In [59]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,6.732000,5.426727,6.619000,0.000000,6.571400,6.714300,16.180000
2,5.186500,3.373900,11.000000,0.000000,11.000000,11.166700,9.900000
3,3.913400,2.013308,11.000000,0.000000,11.000000,11.000000,5.080000
4,2.991800,1.622975,17.000000,0.000000,17.000000,17.000000,4.360000
5,2.223400,1.477527,19.333300,0.000000,19.333300,19.333300,3.880000
6,1.860800,1.389669,19.333300,0.000000,19.333300,19.333300,3.880000
7,1.685700,1.337111,19.333300,0.000000,19.333300,19.333300,3.860000
8,1.681700,1.287255,19.333300,0.000000,19.333300,19.333300,3.840000
9,1.504700,1.256785,17.333300,0.000000,17.333300,17.333300,3.860000
10,1.349700,1.228239,16.666700,0.000000,17.333300,17.333300,3.820000


TrainOutput(global_step=29300, training_loss=1.2324457766171608, metrics={'train_runtime': 10252.2542, 'train_samples_per_second': 2.858, 'train_steps_per_second': 2.858, 'total_flos': 174242833920000.0, 'train_loss': 1.2324457766171608, 'epoch': 50.0})

In [61]:
df.head()

,Polish,English
0,Adam,Adam
1,Adrian,Adrian
2,Albert,Albert
3,Albin,Albin
4,Aleksander,Alexander


In [62]:
from transformers import pipeline

model = pipeline('text2text-generation', model='marcus2000/polish_transliterator2')

In [64]:
for name in dataset['test']['Polish']:
  print('Polish: ', name, 'English: ', model(name)[0]['generated_text'])

Polish:  Żelisław English:  Elisabeth
Polish:  Wyspy Salomona English:  Salomon
Polish:  Ginter English:  Ginter
Polish:  Karina English:  Karina
Polish:  Leo English:  Leon
Polish:  Słowacja English:  Slovenia
Polish:  Dobrosław English:  Dominic
Polish:  Halina English:  Halina
Polish:  Nowa Zelandia English:  Zeland
Polish:  Romuald English:  Romuald
Polish:  Liban English:  Liban
Polish:  Witold English:  William
Polish:  Izrael English:  Isreal
Polish:  Słowenia English:  Slovenia
Polish:  Korea Północna English:  Korea
Polish:  Somalia English:  Somalia
Polish:  Gwinea English:  Georgia
Polish:  Albert English:  Albert
Polish:  Aleksandra English:  Alexandra
Polish:  Rozalia English:  Rozalia
Polish:  Turcja English:  Turcja
Polish:  Bogusław English:  Bulgaria
Polish:  Maria English:  Maria
Polish:  Angela English:  Angela
Polish:  Antonina English:  Antonina
Polish:  Renata English:  Renata
Polish:  Janusz English:  Janus
Polish:  Egipt English:  Egypt
Polish:  Ireneusz English

In [65]:
preds = []

for name in dataset['test']['Polish']:
   preds.append(model(name)[0]['generated_text'])

In [67]:
from sklearn.metrics import accuracy_score

accuracy_score(dataset['test']['English'], preds)

0.34